In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 5.3 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=cc41892e68fa61ff0dfd34750a06732cf0772d283d5c6ced9830f092a3a3180c
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv
 a0529-a0611_US		       dataframe_result_csv_EU
 a0530.csv		       dataframe_result_csv_UK
 a0531.csv		       Github
 a0601.csv		       heatmap529_UK.ipynb
 a0602.csv		       heatmap530.ipynb
 a0603.csv		       Heat_Map_Online_language.ipynb
 ccc_compiled.csv	       __MACOSX
 csv			       mapping_UK
 csv.zip		       mapping_US
 dataframe_csv		       non_violant1.csv
 dataframe_csv_EU	       non_violant2.csv
 dataframe_csv_UK	       UK
 dataframe_result_529_UK.csv   violant.csv
 dataframe_result_530_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_531_UK.csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_601 = 'csv/US06_04.csv'

In [8]:
ct = pd.read_csv(ct_path_601)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,494739473885296_4400310719994799,Facebook,2020-06-04 22:25:11,2021-10-06 01:07:11,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"Without a doubt, the past week has been a roll...",1.0,https://www.facebook.com/494739473885296/photo...,https://www.facebook.com/494739473885296/posts...,2473,3.962025,1.0,149,22,83,59,0,0,0,0,0,15,36,6,8,11,3,4,6,5,0,3,2054267,Nevada City Police Department,NCPDCA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,This is the Official Facebook page for the Nev...,False,2012-07-24 08:33:49,en,0,2054267|4400310719994799
1,121306027893052_3238007812889509,Facebook,2020-06-04 22:25:10,2021-07-27 11:45:05,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"To the family and friends of George Floyd, my ...",1.0,https://www.facebook.com/SolveigMusic/photos/a...,https://www.facebook.com/121306027893052/posts...,8650,3.361446,1.0,103,19,23,125,3,0,6,0,0,38,59,3,6,12,2,0,1,0,0,2,3208429,Solveig Leithaug Music,SolveigMusic,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Solveig Leithaug (Sol-vey Light-houg) is a Dov...,False,2010-05-11 21:42:45,en,0,3208429|3238007812889509
2,1453936424864551_2678215189103329,Facebook,2020-06-04 22:25:07,2021-04-01 23:30:28,link,"Floyd to be eulogized in Minneapolis memorial,...",wsiltv.com,The first of three funeral events for Floyd wa...,A memorial service for George Floyd is being h...,1.0,https://wsiltv.com/2020/06/04/floyd-to-be-eulo...,https://www.facebook.com/1453936424864551/post...,0,-0.705882,1.0,3,1,0,0,0,0,1,0,0,0,4,2,2,2,2,0,2,3,0,1,12994430,Brooke Schlyer TV,BrookeSchlyerTV,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Brooke's journalism page,False,2014-06-16 19:56:38,en,0,12994430|2678215189103329
3,201170940075_10163668330810076,Facebook,2020-06-04 22:25:03,2021-08-23 11:15:06,link,Floyd Mayweather to Cover Funeral Expenses for...,thesource.com,"The Magazine of Hip Hop Music,Culture and Poli...",Floyd Mayweather To Cover Funeral Expenses For...,2.0,http://ow.ly/lqnz30qM8gj,https://www.facebook.com/201170940075/posts/10...,1835799,-6.727273,1.0,4,6,0,1,0,0,0,0,0,0,21,6,12,7,6,12,6,4,0,2,12200,The Source Magazine,TheSourceMagazine,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Fresh Since '88 — All For The Love Of Hip-Hop ...,True,2009-07-02 22:02:18,en,0,12200|10163668330810076
4,463428970501884_1585191744992262,Facebook,2020-06-04 22:25:02,2021-06-28 09:37:19,link,The Rock Asks The Question On Everyone's Mind:...,romper.com,In a passionate plea not-so-subtly directed at...,"In a lengthy video, he repeatedly questioned w...",1.0,https://www.romper.com/p/dwayne-the-rock-johns...,https://www.facebook.com/463428970501884/posts...,895070,2.112500,1.0,133,10,11,12,0,2,0,1,0,4,17,7,15,15,4,6,8,8,0,6,446429,Romper,Romperdotcom,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,What parents are talking about.,True,2015-08-06 15:22:13,en,0,446429|1585191744992262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Nevada County': 1},{}
1,{},{}
2,{'Minneapolis': 1},{'Minneapolis': 1}
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0604.csv')

In [25]:
ct=pd.read_csv('a0604.csv')

In [26]:
!ls

 a0529-a0611_UK		       dataframe_result_530_UK.csv
 a0529-a0611_US		       dataframe_result_531_UK.csv
 a0530.csv		       dataframe_result_csv
 a0531.csv		       dataframe_result_csv_EU
 a0601.csv		       dataframe_result_csv_UK
 a0602.csv		       Github
 a0603.csv		       heatmap529_UK.ipynb
 a0604.csv		       heatmap530.ipynb
 a0605.csv		       Heat_Map_Online_language.ipynb
 a0606.csv		       __MACOSX
 a0607.csv		       mapping_UK
 ccc_compiled.csv	       mapping_US
 csv			       non_violant1.csv
 csv.zip		       non_violant2.csv
 dataframe_csv		       UK
 dataframe_csv_EU	       violant.csv
 dataframe_csv_UK	      '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_529_UK.csv


In [27]:
###
#ct=pd.read_csv('a0601.csv')

In [28]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,494739473885296_4400310719994799,Facebook,2020-06-04 22:25:11,2021-10-06 01:07:11,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"Without a doubt, the past week has been a roll...",1.0,https://www.facebook.com/494739473885296/photo...,https://www.facebook.com/494739473885296/posts...,2473,3.962025,1.0,149,22,83,59,0,0,0,0,0,15,36,6,8,11,3,4,6,5,0,3,2054267,Nevada City Police Department,NCPDCA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,This is the Official Facebook page for the Nev...,False,2012-07-24 08:33:49,en,0,2054267|4400310719994799,{'Nevada County': 1},{},{'Minneapolis': 1}
1,1,121306027893052_3238007812889509,Facebook,2020-06-04 22:25:10,2021-07-27 11:45:05,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"To the family and friends of George Floyd, my ...",1.0,https://www.facebook.com/SolveigMusic/photos/a...,https://www.facebook.com/121306027893052/posts...,8650,3.361446,1.0,103,19,23,125,3,0,6,0,0,38,59,3,6,12,2,0,1,0,0,2,3208429,Solveig Leithaug Music,SolveigMusic,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Solveig Leithaug (Sol-vey Light-houg) is a Dov...,False,2010-05-11 21:42:45,en,0,3208429|3238007812889509,{},{},{'Minneapolis': 1}
2,2,1453936424864551_2678215189103329,Facebook,2020-06-04 22:25:07,2021-04-01 23:30:28,link,"Floyd to be eulogized in Minneapolis memorial,...",wsiltv.com,The first of three funeral events for Floyd wa...,A memorial service for George Floyd is being h...,1.0,https://wsiltv.com/2020/06/04/floyd-to-be-eulo...,https://www.facebook.com/1453936424864551/post...,0,-0.705882,1.0,3,1,0,0,0,0,1,0,0,0,4,2,2,2,2,0,2,3,0,1,12994430,Brooke Schlyer TV,BrookeSchlyerTV,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Brooke's journalism page,False,2014-06-16 19:56:38,en,0,12994430|2678215189103329,{'Minneapolis': 1},{'Minneapolis': 1},{'Minneapolis': 1}
3,3,201170940075_10163668330810076,Facebook,2020-06-04 22:25:03,2021-08-23 11:15:06,link,Floyd Mayweather to Cover Funeral Expenses for...,thesource.com,"The Magazine of Hip Hop Music,Culture and Poli...",Floyd Mayweather To Cover Funeral Expenses For...,2.0,http://ow.ly/lqnz30qM8gj,https://www.facebook.com/201170940075/posts/10...,1835799,-6.727273,1.0,4,6,0,1,0,0,0,0,0,0,21,6,12,7,6,12,6,4,0,2,12200,The Source Magazine,TheSourceMagazine,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Fresh Since '88 — All For The Love Of Hip-Hop ...,True,2009-07-02 22:02:18,en,0,12200|10163668330810076,{},{},{}
4,4,463428970501884_1585191744992262,Facebook,2020-06-04 22:25:02,2021-06-28 09:37:19,link,The Rock Asks The Question On Everyone's Mind:...,romper.com,In a passionate plea not-so-subtly directed at...,"In a lengthy video, he repeatedly questioned w...",1.0,https://www.romper.com/p/dwayne-the-rock-johns...,https://www.facebook.com/463428970501884/posts...,895070,2.112500,1.0,133,10,11,12,0,2,0,1,0,4,17,7,15,15,4,6,8,8,0,6,446429,Romper,Romperdotcom,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,What parents are talking about.,True,2015-08-06 15:22:13,en,0,446429|1585191744992262,{},{},{}


In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
###
ct_ner = ct

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Nevada County': 1},{},{'Minneapolis': 1}
1,{},{},{'Minneapolis': 1}
2,{'Minneapolis': 1},{'Minneapolis': 1},{'Minneapolis': 1}
3,{},{},{}
4,{},{},{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [34]:
ct_ner['NER-msg']

0        [Nevada County]
1                     []
2          [Minneapolis]
3                     []
4                     []
              ...       
24857                 []
24858                 []
24859                 []
24860                 []
24861                 []
Name: NER-msg, Length: 24862, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [36]:
ct_ner['NER']

0                   [Nevada County, Minneapolis]
1                                  [Minneapolis]
2        [Minneapolis, Minneapolis, Minneapolis]
3                                             []
4                                             []
                          ...                   
24857                       [Hartford, Hartford]
24858                                         []
24859                                         []
24860                                         []
24861                                         []
Name: NER, Length: 24862, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [39]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,494739473885296_4400310719994799,Facebook,2020-06-04 22:25:11,2021-10-06 01:07:11,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"Without a doubt, the past week has been a roll...",1.0,https://www.facebook.com/494739473885296/photo...,https://www.facebook.com/494739473885296/posts...,2473,3.962025,1.0,149,22,83,59,0,0,0,0,0,15,36,6,8,11,3,4,6,5,0,3,2054267,Nevada City Police Department,NCPDCA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,This is the Official Facebook page for the Nev...,False,2012-07-24 08:33:49,en,0,2054267|4400310719994799,[Nevada County],[],[Minneapolis],"[nevada county, minneapolis]"
1,1,121306027893052_3238007812889509,Facebook,2020-06-04 22:25:10,2021-07-27 11:45:05,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"To the family and friends of George Floyd, my ...",1.0,https://www.facebook.com/SolveigMusic/photos/a...,https://www.facebook.com/121306027893052/posts...,8650,3.361446,1.0,103,19,23,125,3,0,6,0,0,38,59,3,6,12,2,0,1,0,0,2,3208429,Solveig Leithaug Music,SolveigMusic,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Solveig Leithaug (Sol-vey Light-houg) is a Dov...,False,2010-05-11 21:42:45,en,0,3208429|3238007812889509,[],[],[Minneapolis],[minneapolis]
2,2,1453936424864551_2678215189103329,Facebook,2020-06-04 22:25:07,2021-04-01 23:30:28,link,"Floyd to be eulogized in Minneapolis memorial,...",wsiltv.com,The first of three funeral events for Floyd wa...,A memorial service for George Floyd is being h...,1.0,https://wsiltv.com/2020/06/04/floyd-to-be-eulo...,https://www.facebook.com/1453936424864551/post...,0,-0.705882,1.0,3,1,0,0,0,0,1,0,0,0,4,2,2,2,2,0,2,3,0,1,12994430,Brooke Schlyer TV,BrookeSchlyerTV,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Brooke's journalism page,False,2014-06-16 19:56:38,en,0,12994430|2678215189103329,[Minneapolis],[Minneapolis],[Minneapolis],[minneapolis]
3,3,201170940075_10163668330810076,Facebook,2020-06-04 22:25:03,2021-08-23 11:15:06,link,Floyd Mayweather to Cover Funeral Expenses for...,thesource.com,"The Magazine of Hip Hop Music,Culture and Poli...",Floyd Mayweather To Cover Funeral Expenses For...,2.0,http://ow.ly/lqnz30qM8gj,https://www.facebook.com/201170940075/posts/10...,1835799,-6.727273,1.0,4,6,0,1,0,0,0,0,0,0,21,6,12,7,6,12,6,4,0,2,12200,The Source Magazine,TheSourceMagazine,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Fresh Since '88 — All For The Love Of Hip-Hop ...,True,2009-07-02 22:02:18,en,0,12200|10163668330810076,[],[],[],[]
4,4,463428970501884_1585191744992262,Facebook,2020-06-04 22:25:02,2021-06-28 09:37:19,link,The Rock Asks The Question On Everyone's Mind:...,romper.com,In a passionate plea not-so-subtly directed at...,"In a lengthy video, he repeatedly questioned w...",1.0,https://www.romper.com/p/dwayne-the-rock-johns...,https://www.facebook.com/463428970501884/posts...,895070,2.112500,1.0,133,10,11,12,0,2,0,1,0,4,17,7,15,15,4,6,8,8,0,6,446429,Romper,Romperdotcom,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,What parents are talking about.,True,2015-08-06 15:22:13,en

In [40]:
ct_ner['NER-1']=ct_ner['NER']

In [41]:
ct_ner['NER-1'][0]

['nevada county', 'minneapolis']

In [42]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 13.0 MB/s 


In [43]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [44]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,494739473885296_4400310719994799,Facebook,2020-06-04 22:25:11,2021-10-06 01:07:11,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"Without a doubt, the past week has been a roll...",1.0,https://www.facebook.com/494739473885296/photo...,https://www.facebook.com/494739473885296/posts...,2473,3.962025,1.0,149,22,83,59,0,0,0,0,0,15,36,6,8,11,3,4,6,5,0,3,2054267,Nevada City Police Department,NCPDCA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,This is the Official Facebook page for the Nev...,False,2012-07-24 08:33:49,en,0,2054267|4400310719994799,[Nevada County],[],[Minneapolis],"[nevada county, minneapolis]","[nevada county, minneapolis]"
1,1,121306027893052_3238007812889509,Facebook,2020-06-04 22:25:10,2021-07-27 11:45:05,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"To the family and friends of George Floyd, my ...",1.0,https://www.facebook.com/SolveigMusic/photos/a...,https://www.facebook.com/121306027893052/posts...,8650,3.361446,1.0,103,19,23,125,3,0,6,0,0,38,59,3,6,12,2,0,1,0,0,2,3208429,Solveig Leithaug Music,SolveigMusic,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Solveig Leithaug (Sol-vey Light-houg) is a Dov...,False,2010-05-11 21:42:45,en,0,3208429|3238007812889509,[],[],[Minneapolis],[minneapolis],[minneapolis]
2,2,1453936424864551_2678215189103329,Facebook,2020-06-04 22:25:07,2021-04-01 23:30:28,link,"Floyd to be eulogized in Minneapolis memorial,...",wsiltv.com,The first of three funeral events for Floyd wa...,A memorial service for George Floyd is being h...,1.0,https://wsiltv.com/2020/06/04/floyd-to-be-eulo...,https://www.facebook.com/1453936424864551/post...,0,-0.705882,1.0,3,1,0,0,0,0,1,0,0,0,4,2,2,2,2,0,2,3,0,1,12994430,Brooke Schlyer TV,BrookeSchlyerTV,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Brooke's journalism page,False,2014-06-16 19:56:38,en,0,12994430|2678215189103329,[Minneapolis],[Minneapolis],[Minneapolis],[minneapolis],[minneapolis]
3,3,201170940075_10163668330810076,Facebook,2020-06-04 22:25:03,2021-08-23 11:15:06,link,Floyd Mayweather to Cover Funeral Expenses for...,thesource.com,"The Magazine of Hip Hop Music,Culture and Poli...",Floyd Mayweather To Cover Funeral Expenses For...,2.0,http://ow.ly/lqnz30qM8gj,https://www.facebook.com/201170940075/posts/10...,1835799,-6.727273,1.0,4,6,0,1,0,0,0,0,0,0,21,6,12,7,6,12,6,4,0,2,12200,The Source Magazine,TheSourceMagazine,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Fresh Since '88 — All For The Love Of Hip-Hop ...,True,2009-07-02 22:02:18,en,0,12200|10163668330810076,[],[],[],[],[]
4,4,463428970501884_1585191744992262,Facebook,2020-06-04 22:25:02,2021-06-28 09:37:19,link,The Rock Asks The Question On Everyone's Mind:...,romper.com,In a passionate plea not-so-subtly directed at...,"In a lengthy video, he repeatedly questioned w...",1.0,https://www.romper.com/p/dwayne-the-rock-johns...,https://www.facebook.com/463428970501884/posts...,895070,2.112500,1.0,133,10,11,12,0,2,0,1,0,4,17,7,15,15,4,6,8,8,0,6,446429,Romper,Romperdotcom,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook

In [45]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [46]:
!ls

dataframe0529_post.csv	dataframe_mapping_0529.csv  dataframe_mapping_0605.csv
dataframe0530_post.csv	dataframe_mapping_0530.csv  dataframe_mapping_0606.csv
dataframe0531_post.csv	dataframe_mapping_0531.csv  dataframe_mapping_0607.csv
dataframe0602_post.csv	dataframe_mapping_0601.csv  dataframe_mapping_0608.csv
dataframe0603_post.csv	dataframe_mapping_0602.csv  dataframe_mapping_0609.csv
dataframe0606_post.csv	dataframe_mapping_0603.csv  dataframe_mapping_0610.csv
dataframe0607_post.csv	dataframe_mapping_0604.csv  dataframe_mapping_0611.csv


In [47]:
ct_mapping=pd.read_csv('dataframe_mapping_0604.csv',encoding='ISO-8859-1')

In [48]:
len(ct_mapping['City'])

2834

In [49]:
len(ct_ner['NER-1'])

24862

In [50]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [51]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,494739473885296_4400310719994799,Facebook,2020-06-04 22:25:11,2021-10-06 01:07:11,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"Without a doubt, the past week has been a roll...",1.0,https://www.facebook.com/494739473885296/photo...,https://www.facebook.com/494739473885296/posts...,2473,3.962025,1.0,149,22,83,59,0,0,0,0,0,15,36,6,8,11,3,4,6,5,0,3,2054267,Nevada City Police Department,NCPDCA,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,This is the Official Facebook page for the Nev...,False,2012-07-24 08:33:49,en,0,2054267|4400310719994799,[Nevada County],[],[Minneapolis],"[nevada county, minneapolis]","[Nevada County, California, United States, Min..."
1,1,121306027893052_3238007812889509,Facebook,2020-06-04 22:25:10,2021-07-27 11:45:05,photo,'Get your knee off our necks!': George Floyd m...,wvtm13.com,People gathered in front of the casket of Geor...,"To the family and friends of George Floyd, my ...",1.0,https://www.facebook.com/SolveigMusic/photos/a...,https://www.facebook.com/121306027893052/posts...,8650,3.361446,1.0,103,19,23,125,3,0,6,0,0,38,59,3,6,12,2,0,1,0,0,2,3208429,Solveig Leithaug Music,SolveigMusic,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Solveig Leithaug (Sol-vey Light-houg) is a Dov...,False,2010-05-11 21:42:45,en,0,3208429|3238007812889509,[],[],[Minneapolis],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, Unit..."
2,2,1453936424864551_2678215189103329,Facebook,2020-06-04 22:25:07,2021-04-01 23:30:28,link,"Floyd to be eulogized in Minneapolis memorial,...",wsiltv.com,The first of three funeral events for Floyd wa...,A memorial service for George Floyd is being h...,1.0,https://wsiltv.com/2020/06/04/floyd-to-be-eulo...,https://www.facebook.com/1453936424864551/post...,0,-0.705882,1.0,3,1,0,0,0,0,1,0,0,0,4,2,2,2,2,0,2,3,0,1,12994430,Brooke Schlyer TV,BrookeSchlyerTV,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Brooke's journalism page,False,2014-06-16 19:56:38,en,0,12994430|2678215189103329,[Minneapolis],[Minneapolis],[Minneapolis],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, Unit..."
3,3,201170940075_10163668330810076,Facebook,2020-06-04 22:25:03,2021-08-23 11:15:06,link,Floyd Mayweather to Cover Funeral Expenses for...,thesource.com,"The Magazine of Hip Hop Music,Culture and Poli...",Floyd Mayweather To Cover Funeral Expenses For...,2.0,http://ow.ly/lqnz30qM8gj,https://www.facebook.com/201170940075/posts/10...,1835799,-6.727273,1.0,4,6,0,1,0,0,0,0,0,0,21,6,12,7,6,12,6,4,0,2,12200,The Source Magazine,TheSourceMagazine,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Fresh Since '88 — All For The Love Of Hip-Hop ...,True,2009-07-02 22:02:18,en,0,12200|10163668330810076,[],[],[],[],[]
4,4,463428970501884_1585191744992262,Facebook,2020-06-04 22:25:02,2021-06-28 09:37:19,link,The Rock Asks The Question On Everyone's Mind:...,romper.com,In a passionate plea not-so-subtly directed at...,"In a lengthy video, he repeatedly questioned w...",1.0,https://www.romper.com/p/dwayne-the-rock-johns...,https://www.facebook.com/463428970501884/posts...,895070,2.112500,1.0,133,10,11,12,0,2,0,1,0,4,17,7,15,15,

In [52]:
ct_ner['NER-1'][12]

[]

In [53]:
ct_ner.to_csv('dataframe0604_post.csv')